In [ ]:
# %%capture
# !pip install webdriver-manager
# !pip install selenium
# !pip install requests
# !pip install bs4
# !pip install mitmproxy
# !pip install fake-useragent
# # install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install selenium
# !pip install tqdl

In [ ]:
# import re
# import os
# import csv
# import time
# import requests

# from time import sleep
# from tqdm import tqdm, trange
# from bs4 import BeautifulSoup
# from selenium import webdriver 
# from fake_useragent import UserAgent
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.support.ui import WebDriverWait
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support import expected_conditions as EC
# #try
# from selenium.common.exceptions import NoSuchElementException
# from selenium.common.exceptions import StaleElementReferenceException
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions



In [ ]:
# option = webdriver.ChromeOptions()
# option.add_argument('--headless')
# option.add_argument('--no-sandbox')
# option.add_argument('--disable-dev-shm-usage')
# option.add_argument('--disable-blink-features=AutomationControlled')  # 无头模式，防检测
# #option.manage().timeouts().implicitlyWait(2000);
# # option.manage().timeouts().pageLoadTimeout(40);
# # option.manage().timeouts().setScriptTimeout(60);
# option.add_argument('window-size=1920x1080')
# option.add_argument('log-level=3')
# option.add_argument("--user-agent=*****")
# option.add_argument("--disable-dev-shm-usage")
# option.add_argument("--no-sandbox")
# option.add_argument("lang=zh_CN.UTF-8")
# option.add_argument('--disable-gpu')
# option.add_argument('--incognito')
# option.add_argument("--window-size=1920,1050")  # 专门应对无头浏览器中不能最大化屏幕的方案
# option.add_experimental_option('excludeSwitches', ['enable-automation'])
# option.add_experimental_option('useAutomationExtension', False)

In [4]:
import pandas as pd

# open it, go to a website, and get results
df = pd.read_excel("content/uspto_test.xlsx")
df.head()

,品牌名稱,公司名稱,統編,電話,地址,URL
0,"TAIWAN FU HSING INDUSTRIAL CO., LTD.",NaN,NaN,NaN,NaN,NaN
1,"TAIWAN SEMICONDUCTOR MANUFACTURING CO, LTD.",NaN,NaN,NaN,NaN,NaN
2,TAIWAN BIFIDO FOODS INC. CORPORATION,NaN,NaN,NaN,NaN,NaN
3,TAIWAN KENLEN SEWING MACHINE DEVELOPMENT COMPANY,NaN,NaN,NaN,NaN,NaN
4,"Accuratech Automation CO., LTD.",NaN,NaN,NaN,NaN,NaN


In [6]:
# Catch the first col
CompList = df.iloc[:,0].tolist()  
CompList

print("[ The Length of Website to Crawl:", len(CompList), "]\n")

[ The Length of Website to Crawl: 249 ]



In [11]:
serachKey = []

for i in CompList:
    try: 
        serachKey.append(i + " 台灣公司網")
    except:
        serachKey.append(str(i) + " 台灣公司網")

print(serachKey[0])

TAIWAN FU HSING INDUSTRIAL CO., LTD. 台灣公司網


In [4]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

wd = webdriver.Chrome(ChromeDriverManager().install())

ModuleNotFoundError: No module named 'webdriver_manager'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By

ans = []
wd = webdriver.Chrome(executable_path='driver/chromedriver')

# wd = webdriver.Chrome('chromedriver', options=option)  # 把参数配置设置到浏览器中
# waitWebDriver = WebDriverWait (wd, 250)  


/var/folders/9q/486czkcn7lv5v0hwbt71twdc0000gn/T/ipykernel_6775/4204272214.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path='driver/chromedriver')


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [ ]:


for i in tqdm(serachKey):
    #launch URL
    wd.get("https://www.google.com/")
    #identify search box
    m = wd.find_element(By.NAME, "q")
    #enter search text
    #print(i)
    m.send_keys(i)

    #perform Google search with Keys.ENTER
    m.send_keys(Keys.ENTER)
    
    allResultTitle = wd.find_elements(By.TAG_NAME, 'h3')

    #print(allResultLink)

    result = ""

    trynum = 0
    while( trynum< len(allResultTitle)):
        try:
            allResultTitle[trynum].click()
        except:
            allResultTitle = wd.find_elements(By.TAG_NAME, 'h3')
            trynum += 1
            continue

        if "twincn.com" in wd.current_url:
            wd.back()
            allResultTitle = wd.find_elements(By.TAG_NAME, 'h3')
            result = allResultTitle[trynum]
            #print("right")
            break
        else:
            wd.back()
            allResultTitle = wd.find_elements(By.TAG_NAME, 'h3')
            trynum += 1

    try:
        result.click()
    except:
        ans.append(["","","","",""])
        print("Can't Find Company")
        continue
    

    soup = BeautifulSoup(wd.page_source,'html.parser')
    items = soup.find_all('div',{'class':'contents text-center'})
    #print(items)

    if (items):
        for each in items:
            try:
                head = each.find('h1').text.replace(" ", "").replace("\n", "")

                brs = each.find('p').text
                
                BrsSplit = re.split('\n| ', brs)
                BrsSplit = [i.replace(":", "").replace(",", "") for i in BrsSplit if (i != "" and i != "\n") ]
                BrsSplit = [i.replace("No.", "").replace("代表人", "").replace("傳真", "") for i in BrsSplit if ("股票代號" not in i and "Inc" not in i and ("代表人" in i or "傳真" in i or "路" in i or "號" in i))]
                #print(BrsSplit)
                BrsSplit[2] = BrsSplit[2].replace("(", "").replace(")", "").replace("-", "")
                
                posAddStart = 0
                for i, item in enumerate(BrsSplit[2]):
                    if not item.isdigit():
                        posAddStart = i
                        break
                #print(BrsSplit[2][posAddStart:])

                addressEnd = BrsSplit[2].find(".")

                BrsSplit[2] = BrsSplit[2][posAddStart:] if addressEnd == -1 else BrsSplit[2][posAddStart:addressEnd]

                res = [head, BrsSplit[0], BrsSplit[1], BrsSplit[2], wd.current_url]

                print(res)
                ans.append(res)

            except:
                try: 
                    toAppend = ["","","","",""]
                    head = each.find('h1').text.replace(" ", "").replace("\n", "")

                    brs = each.find('p').text
                    BrsSplit = re.split('\n| ', brs)
                    BrsSplit = [i.replace(":", "").replace(",", "") for i in BrsSplit if (i != "" and i != "\n") ]
                    BrsSplit = [i.replace("No.", "").replace("代表人", "").replace("傳真", "") for i in BrsSplit if ("股票代號" not in i and "Inc" not in i and ("代表人" in i or "傳真" in i or "路" in i or "號" in i))]
                    BrsSplit

                    city = ["臺北市", "新北市", "桃園市", "臺中市", "臺南市", "高雄市", "宜蘭縣", "新竹縣", 
                            "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣", "屏東縣", "花蓮縣", "臺東縣", 
                            "澎湖縣", "基隆市", "新竹市", "嘉義市"]
                    
                    address = ""
                    idNumber = ""

                    for item in BrsSplit:
                        for cityItem in city:
                            if cityItem in item:
                                addressStart = item.find(cityItem)
                                addressEnd = item.find(".")
                                address = item[addressStart:] if addressEnd == -1 else item[addressStart:addressEnd]
                                idNumber = item[:addressStart]
                                break 

                    # Company Name
                    toAppend[0] = head
                    # tax ID number
                    toAppend[1] = idNumber if idNumber.isdigit() else BrsSplit[0]
                    # Company Url
                    toAppend[-1] = wd.current_url
                    # Company Address
                    toAppend[-2] = address

                    print(toAppend)
                    ans.append(toAppend)
                except:
                    toAppend = ["","","","",""]
                    head = each.find('h1').text.replace(" ", "").replace("\n", "")

                    brs = each.find('p').text
                    BrsSplit = re.split('\n| ', brs)
                    BrsSplit = [i.replace(":", "").replace(",", "") for i in BrsSplit if (i != "" and i != "\n") ]
                    BrsSplit = [i.replace("No.", "").replace("代表人", "").replace("傳真", "") for i in BrsSplit if ("股票代號" not in i and "Inc" not in i and ("代表人" in i or "傳真" in i or "路" in i or "號" in i))]
                    BrsSplit
                    #print(BrsSplit, "!!!!!")
                    print("Info Struct Incorrect")
                    ans.append(["","","","",""])
    else:
        print("Info Not Avaliable")
        ans.append(["","","","",""])

FileNotFoundError: ignored

In [ ]:
column_names = ["公司名稱", "統編", "電話", "地址", "URL"]
data_df = pd.DataFrame(ans, columns=column_names)
data_df.insert(0, '品牌名稱', CompList[:])
data_df.to_excel("output.xlsx")  